In [1]:
# 필요한 모듈 및 라이브러리 불러오기
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import random
import sys
import re
import requests
from bs4 import BeautifulSoup

In [2]:
selenium.__version__

'3.141.0'

In [3]:
print(sys.version)

3.7.7 (default, Mar 23 2020, 17:31:31) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [214]:
drink = pd.DataFrame(columns=['id','name','category','calories(kcal)','sugars(g)','protein(g)',
                              'sodium(mg)','saturated_fat(mg)','caffeine(mg)','allergy'])

In [226]:
drink

,id,name,category,calories(kcal),sugars(g),protein(g),sodium(mg),saturated_fat(mg),caffeine(mg),allergy


In [227]:
# 크롬 브라우저 실행

options = webdriver.ChromeOptions()
# 크롬 창이 켜지지 않은 상태로 진행하고 싶은 경우 아래 옵션 수행
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--start-maximized")

driver = webdriver.Chrome('/Users/JGH/chromedriver_mac_arm64/chromedriver', options=options)

In [228]:
url = "https://www.starbucks.co.kr/menu/drink_list.do"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
coffee_info = soup.find_all(class_='coffeeInfo mb60')

# 카테고리 별 상품 수
category_count = [14,2,36,10,11,4,3,35,9,17]

# 드라이버 실행
driver.get(url)

In [ ]:
product_id = 1000
for i in range(len(coffee_info)):
    category = coffee_info[i]['summary'].replace(' 영양정보', '')
    for j in range(category_count[i]):
        element = driver.find_element_by_xpath(f'//*[@id="container"]/div[2]/div[2]/div/dl/dd[1]/div[1]/dl/dd[{i+1}]/ul/li[{j+1}]')
        element.click()
        product_id += 1
        data = {
            'id': product_id,
            'name': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/div[1]/h4').text.split('\n')[0],
            'category': category,
            'calories(kcal)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[1]/dl/dd').text,
            'sugars(g)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[2]/dl/dd').text,
            'protein(g)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[3]/dl/dd').text,
            'sodium(mg)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[1]/dl/dd').text,
            'saturated_fat(mg)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[2]/dl/dd').text,
            'caffeine(mg)': driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[3]/dl/dd').text,
            'allergy' : driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[3]/p').text.replace('알레르기 유발요인 : ','')
        }

        drink = drink.append(data, ignore_index=True)
        driver.back()

In [271]:
drink

,id,name,category,calories(kcal),sugars(g),protein(g),sodium(mg),saturated_fat(mg),caffeine(mg),allergy
0,1001,리저브 나이트로,콜드 브루 커피,5,0,0,0,0,190,
1,1002,나이트로 바닐라 크림,콜드 브루 커피,80,10,1,40,2,232,우유
2,1003,나이트로 콜드 브루,콜드 브루 커피,5,0,0,5,0,245,
3,1004,돌체 콜드 브루,콜드 브루 커피,265,29,8,130,9,155,우유
4,1005,리저브 콜드 브루,콜드 브루 커피,5,0,0,0,0,190,
...,...,...,...,...,...,...,...,...,...,...
136,1137,햇사과 주스 591ML,스타벅스 주스(병음료),360,84,1.3,20,0,0,
137,1138,딸기 가득 요거트 190ML,스타벅스 주스(병음료),155,21,4,50,3.2,0,우유
138,1139,블루베리 요거트 190ML,스타벅스 주스(병음료),155,18,5,70,3.6,0,우유
139,1140,애플망고 코코넛 190ML,스타벅스 주스(병음료),244,21,3,9,11,0,대두


In [272]:
driver.quit()

In [273]:
drink.to_csv('/Users/JGH/Python/Section Project 4/drink.csv', index=False)